In [74]:
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import numpy as np  # Importa numpy per la gestione delle trasformazioni

# Percorsi dei file Excel
f1 = r"C:\Users\Mario\Desktop\3 ANNO\2 SEMESTRE\attaccanti\attaccanti_19.xlsx"
f2 = r"C:\Users\Mario\Desktop\3 ANNO\2 SEMESTRE\attaccanti\attaccanti_20.xlsx"
f3 = r"C:\Users\Mario\Desktop\3 ANNO\2 SEMESTRE\attaccanti\attaccanti_21.xlsx"
f4 = r"C:\Users\Mario\Desktop\3 ANNO\2 SEMESTRE\attaccanti\attaccanti_22.xlsx"
f5 = r"C:\Users\Mario\Desktop\3 ANNO\2 SEMESTRE\attaccanti\attaccanti_23.xlsx"

# Colonne da eliminare
colonne_da_eliminare = ['Player', 'Nation', 'Pos', 'Squad', 'Age', 'MP', 'Starts', 'Min', 
                        'G+A', 'G-PK', 'PK', 'PKatt', 'CrdY', 'CrdR', 'PrgP',
                        'SoT%', 'G/Sh', 'G/SoT', 'Dist', 'FK', 'G-xG',
                        'Cmp', 'Att', 'Cmp%', 'CrsPA', 'Tkl',
                        'TklW', 'Mid 3rd', 'Att 3rd', 'Mid 3rd.1',
                        'Succ%', 'TotDist', '2CrdY', 'Fld', 'Off',
                        'Crs', 'Won', 'Lost', 'Won%']

# Leggi e unisci i file Excel
df1 = pd.read_excel(f1)
df2 = pd.read_excel(f2).iloc[1:].reset_index(drop=True)
df3 = pd.read_excel(f3).iloc[1:].reset_index(drop=True)
df4 = pd.read_excel(f4).iloc[1:].reset_index(drop=True)
df_unito = pd.concat([df1, df2, df3, df4], ignore_index=True)

# Rimuovi le colonne specificate dal DataFrame unito
df_unito = df_unito.drop(columns=colonne_da_eliminare, errors='ignore')

# Carica il file attaccanti_23.xlsx e rimuovi le stesse colonne
df5 = pd.read_excel(f5).iloc[1:].reset_index(drop=True)
df5 = df5.drop(columns=colonne_da_eliminare, errors='ignore')

# Separare le colonne in input e output
X = df_unito.iloc[:, 2:]  # Tutte le colonne eccetto le prime due
y = df_unito.iloc[:, :2]  # Le prime due colonne

# Pre-elaborazione
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Divisione in set di addestramento e test
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Costruzione della rete neurale
model = tf.keras.Sequential([
    tf.keras.layers.Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(y_train.shape[1])
])

# Compilazione del modello
model.compile(optimizer='adam', loss='mean_squared_error')

# Addestramento del modello
model.fit(X_train, y_train, epochs=3000, batch_size=32, validation_split=0.2)

# Carica il nuovo file e pre-processa i dati
df_new = df5  # Usa il DataFrame df5 che abbiamo già elaborato

# Verifica che le colonne siano allineate con il DataFrame di addestramento
X_new = df_new[X.columns]

# Pre-elabora i dati nuovi (normalizza con lo stesso scaler usato per l'addestramento)
X_new_scaled = scaler.transform(X_new)

# Usa il modello per fare previsioni
y_pred = model.predict(X_new_scaled)

# Imposta a 0 tutti i valori predetti che sono minori di 0
y_pred = np.clip(y_pred, 0, None)  # Sostituisci i valori negativi con 0

# Visualizza le previsioni
df_new['Pred_Gls'] = y_pred[:, 0]  # Previsioni per la prima colonna (Gls)
df_new['Pred_Ast'] = y_pred[:, 1]  # Previsioni per la seconda colonna (Ast)

print(df_new[['Gls', 'Ast', 'Pred_Gls', 'Pred_Ast']])

# Calcola la somma delle previsioni per Gls e Ast
somma_goal_assist_previsti = df_new['Pred_Gls'].sum() + df_new['Pred_Ast'].sum()

somma_goal_assist_veri = df_new['Gls'].sum() + df_new['Ast'].sum()

# Visualizza la somma
print(f"La somma dei goal e assist ottenuti dalla rete neurale è: {somma_goal_assist_previsti}")
print(f"La somma dei goal e assist reali: {somma_goal_assist_veri}")


Epoch 1/3000


C:\Users\Mario\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


15/15 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - loss: 22.7071 - val_loss: 19.4295
Epoch 2/3000
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 13.6325 - val_loss: 12.4258
Epoch 3/3000
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 8.3225 - val_loss: 7.8229
Epoch 4/3000
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 4.4501 - val_loss: 5.7500
Epoch 5/3000
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 4.0467 - val_loss: 4.2505
Epoch 6/3000
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 2.9100 - val_loss: 3.5498
Epoch 7/3000
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 2.9572 - val_loss: 3.2476
Epoch 8/3000
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 2.5387 - val_loss: 2.8834
Epoch 9/3000
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 2.2946 - val_loss: 2.9880
Epoch 10/3000
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 2.2047 - val_loss: 2.7111
Epoch 11/3000
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 2.2827 - val_loss: 2.6996
Epoch 12/3000
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 

Epoch 69/3000
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.9680 - val_loss: 1.8830
Epoch 70/3000
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.9643 - val_loss: 1.8388
Epoch 71/3000
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 1.1351 - val_loss: 1.9759
Epoch 72/3000
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 1.0206 - val_loss: 1.9107
Epoch 73/3000
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.9803 - val_loss: 1.9040
Epoch 74/3000
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 1.1037 - val_loss: 1.9429
Epoch 75/3000
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 1.0394 - val_loss: 1.9095
Epoch 76/3000
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.9320 - val_loss: 1.9107
Epoch 77/3000
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.9847 - val_loss: 1.7998
Epoch 78/3000
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 1.0182 - val_loss: 2.0263
Epoch 79/3000
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.9900 - val_loss: 1.8551
Epoch 80/3000
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s

15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.8632 - val_loss: 1.8048
Epoch 137/3000
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.8078 - val_loss: 2.0610
Epoch 138/3000
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.8077 - val_loss: 1.8726
Epoch 139/3000
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.7656 - val_loss: 1.8656
Epoch 140/3000
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.7427 - val_loss: 1.9674
Epoch 141/3000
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.8519 - val_loss: 1.9726
Epoch 142/3000
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.7190 - val_loss: 1.9152
Epoch 143/3000
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.7762 - val_loss: 1.8873
Epoch 144/3000
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.7378 - val_loss: 1.9809
Epoch 145/3000
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.6808 - val_loss: 1.8628
Epoch 146/3000
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.7156 - val_loss: 1.9046
Epoch 147/3000
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 3m

...

15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0218 - val_loss: 3.0620
Epoch 2724/3000
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0186 - val_loss: 3.0396
Epoch 2725/3000
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0141 - val_loss: 3.0813
Epoch 2726/3000
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0184 - val_loss: 3.0413
Epoch 2727/3000
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0204 - val_loss: 3.0227
Epoch 2728/3000
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0164 - val_loss: 3.0738
Epoch 2729/3000
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0192 - val_loss: 3.0344
Epoch 2730/3000
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0210 - val_loss: 3.0735
Epoch 2731/3000
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0210 - val_loss: 3.0572
Epoch 2732/3000
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0146 - val_loss: 3.0216
Epoch 2733/3000
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0117 - val_loss: 3.0585
Epoch 2734/3000
15/15 ━━━━━━━━━━━━━━━

15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0553 - val_loss: 3.0524
Epoch 2790/3000
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0368 - val_loss: 3.0835
Epoch 2791/3000
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0522 - val_loss: 3.2030
Epoch 2792/3000
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0529 - val_loss: 2.9971
Epoch 2793/3000
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0433 - val_loss: 3.0341
Epoch 2794/3000
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0358 - val_loss: 3.0758
Epoch 2795/3000
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0238 - val_loss: 3.0012
Epoch 2796/3000
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0243 - val_loss: 3.1124
Epoch 2797/3000
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0323 - val_loss: 3.0991
Epoch 2798/3000
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0216 - val_loss: 3.0832
Epoch 2799/3000
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0197 - val_loss: 3.0786
Epoch 2800/3000
15/15 ━━━━━━━━━━━━━━━

15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0164 - val_loss: 3.0520
Epoch 2856/3000
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0144 - val_loss: 3.0755
Epoch 2857/3000
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0182 - val_loss: 3.1205
Epoch 2858/3000
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0297 - val_loss: 3.0824
Epoch 2859/3000
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0215 - val_loss: 3.1005
Epoch 2860/3000
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0193 - val_loss: 3.0826
Epoch 2861/3000
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0215 - val_loss: 3.0900
Epoch 2862/3000
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0298 - val_loss: 3.0757
Epoch 2863/3000
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0174 - val_loss: 3.0944
Epoch 2864/3000
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0215 - val_loss: 3.0502
Epoch 2865/3000
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0162 - val_loss: 3.0378
Epoch 2866/3000
15/15 ━━━━━━━━━━━━━━━

15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0245 - val_loss: 3.0893
Epoch 2922/3000
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0181 - val_loss: 3.1150
Epoch 2923/3000
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0195 - val_loss: 3.0848
Epoch 2924/3000
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0145 - val_loss: 3.0682
Epoch 2925/3000
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0170 - val_loss: 3.0648
Epoch 2926/3000
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0130 - val_loss: 3.0685
Epoch 2927/3000
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0115 - val_loss: 3.0483
Epoch 2928/3000
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0126 - val_loss: 3.0771
Epoch 2929/3000
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0117 - val_loss: 3.1061
Epoch 2930/3000
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0139 - val_loss: 3.0559
Epoch 2931/3000
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0135 - val_loss: 3.0494
Epoch 2932/3000
15/15 ━━━━━━━━━━━━━━━

15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0177 - val_loss: 3.0512
Epoch 2988/3000
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0157 - val_loss: 3.0570
Epoch 2989/3000
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0163 - val_loss: 3.0575
Epoch 2990/3000
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0108 - val_loss: 3.0549
Epoch 2991/3000
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0156 - val_loss: 3.0693
Epoch 2992/3000
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0136 - val_loss: 3.0489
Epoch 2993/3000
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0203 - val_loss: 3.0872
Epoch 2994/3000
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0205 - val_loss: 3.0501
Epoch 2995/3000
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0152 - val_loss: 3.0762
Epoch 2996/3000
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0151 - val_loss: 3.0916
Epoch 2997/3000
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0136 - val_loss: 3.0634
Epoch 2998/3000
15/15 ━━━━━━━━━━━━━━━

In [73]:
#GOAL E ASSIST VERI

y = df5 # Le prime due colonne

somma_colonna_1 = y.iloc[:, 0].sum()  # Somma della prima colonna (ad esempio, Gls)
somma_colonna_2 = y.iloc[:, 1].sum()  # Somma della seconda colonna (ad esempio, Ast)

# Somma totale delle due colonne
somma_totale = somma_colonna_1 + somma_colonna_2

# Visualizza i risultati
print(f"La somma della prima colonna (es. Gls) è: {somma_colonna_1}")
print(f"La somma della seconda colonna (es. Ast) è: {somma_colonna_2}")
print(f"La somma totale di goal e assist è: {somma_totale}")

La somma della prima colonna (es. Gls) è: 622
La somma della seconda colonna (es. Ast) è: 289
La somma totale di goal e assist è: 911


Dati di addestramento (input): Tutte le colonne da df_unito a partire dalla terza colonna in poi, dopo aver eliminato le colonne non pertinenti (come nome, età, squadra, ecc.).

Dati di addestramento (output): Le prime due colonne di df_unito, che probabilmente rappresentano Gls (goal) e Ast (assist).
Dati utilizzati per la previsione: Le stesse colonne di input del file attaccanti_23.xlsx (df5), preprocessati e normalizzati allo stesso modo. Le previsioni forniscono i valori per Gls e Ast di questo nuovo dataset.

Il codice alla fine calcola e stampa la somma dei goal e degli assist previsti dalla rete neurale per i dati di attaccanti_23.xlsx.